In [1]:
#Dependencies
import pandas as pd
import numpy as np
from scipy.stats import poisson, skellam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [2]:
#load in datasets
matches = pd.read_csv("epl_matches.csv", index_col=0) 
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,Match Report,NaN,7.0,1.0,18.9,1.0,0,0,2019,Huddersfield Town
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,Match Report,NaN,13.0,3.0,18.1,1.0,0,0,2019,Huddersfield Town
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,Match Report,NaN,5.0,1.0,21.6,0.0,0,0,2019,Huddersfield Town
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,Match Report,NaN,7.0,3.0,19.0,1.0,0,0,2019,Huddersfield Town


In [3]:
# Get the shape
matches.shape

(4366, 27)

In [4]:
# How many Matches each team has played. Keep in mind of relegation and promotion rules.
matches["team"].value_counts()

team
West Ham United             219
Arsenal                     218
Crystal Palace              218
Manchester City             218
Everton                     218
Tottenham Hotspur           218
Manchester United           218
Brighton and Hove Albion    218
Wolverhampton Wanderers     218
Newcastle United            218
Liverpool                   218
Chelsea                     217
Leicester City              190
Southampton                 190
Aston Villa                 181
Burnley                     181
Fulham                      143
Bournemouth                 142
Watford                     114
Leeds United                114
Brentford                   105
Sheffield United            104
Norwich City                 76
Nottingham Forest            67
West Bromwich Albion         38
Cardiff City                 38
Huddersfield Town            38
Luton Town                   29
Name: count, dtype: int64

In [5]:
# Datatype of each value
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [6]:
#convert date to date-time
matches["date"] = pd.to_datetime(matches["date"])
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
season                   int64
team                    object
dtype: object

In [7]:
# setting the venue code

matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches
# 0 = away 
# 1 = home

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,notes,sh,sot,dist,fk,pk,pkatt,season,team,venue_code
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal,1
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal,0
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal,1
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal,1
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,NaN,7.0,1.0,18.9,1.0,0,0,2019,Huddersfield Town,0
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,NaN,13.0,3.0,18.1,1.0,0,0,2019,Huddersfield Town,1
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,NaN,5.0,1.0,21.6,0.0,0,0,2019,Huddersfield Town,0
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,NaN,7.0,3.0,19.0,1.0,0,0,2019,Huddersfield Town,1


In [8]:
# Create numeric codes for each unique 'opponent' value and store them in a new column 'opp_code'.
matches["opp_code"] = matches["opponent"].astype("category").cat.codes


In [9]:
# Extract the hour component from the 'time' column and store it as integers in a new column named 'hour'.
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,sot,dist,fk,pk,pkatt,season,team,venue_code,opp_code,hour
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,7.0,19.1,0.0,0,0,2024,Arsenal,1,20,12
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,2.0,16.4,0.0,1,1,2024,Arsenal,0,8,20
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,9.0,13.8,0.0,1,1,2024,Arsenal,1,10,15
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,5.0,15.0,0.0,0,0,2024,Arsenal,1,17,16
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,4.0,17.4,0.0,0,0,2024,Arsenal,0,9,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,1.0,18.9,1.0,0,0,2019,Huddersfield Town,0,23,12
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,3.0,18.1,1.0,0,0,2019,Huddersfield Town,1,24,15
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,1.0,21.6,0.0,0,0,2019,Huddersfield Town,0,14,20
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,3.0,19.0,1.0,0,0,2019,Huddersfield Town,1,17,14


In [35]:
# Add a new column 'day_code' to the 'matches' DataFrame, containing the day of the week from the 'date' column.

matches["day_code"] = matches["date"].dt.dayofweek
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,20,12,5,1.0
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,8,20,0,1.0
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,0.0,1,1,2024,Arsenal,1,10,15,5,0.0
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,17,16,6,1.0
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,0.0,0,0,2024,Arsenal,0,9,16,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,1.0,0,0,2019,Huddersfield Town,0,23,12,5,-1.0
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,1.0,0,0,2019,Huddersfield Town,1,24,15,5,-1.0
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,0.0,0,0,2019,Huddersfield Town,0,14,20,4,-1.0
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,1.0,0,0,2019,Huddersfield Town,1,17,14,6,0.0


In [11]:
# target will be if team won lost or drawn
conditions = [
    (matches["result"] == "W"),  # Win condition
    (matches["result"] == "L"),  # Lose condition
    (matches["result"] == "D")   # Draw condition
]

values = [1, -1, 0]  # 1 for win, -1 for lose, 0 for draw
matches["target"] = np.select(conditions, values, default=np.nan)
# Apply np.select to create the 'target' column based on the defined conditions and values
matches


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,20,12,5,1.0
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,8,20,0,1.0
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,0.0,1,1,2024,Arsenal,1,10,15,5,0.0
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,17,16,6,1.0
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,0.0,0,0,2024,Arsenal,0,9,16,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,1.0,0,0,2019,Huddersfield Town,0,23,12,5,-1.0
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,1.0,0,0,2019,Huddersfield Town,1,24,15,5,-1.0
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,0.0,0,0,2019,Huddersfield Town,0,14,20,4,-1.0
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,1.0,0,0,2019,Huddersfield Town,1,17,14,6,0.0


In [12]:
#random forest classifier
# Define the list of predictors including venue code, opponent code, hour, and day code.
predictors = ["venue_code", "opp_code", "hour", "day_code"]

# Define the features (predictors) and the target variable
X = matches[predictors]
y = matches["target"]

# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Initialize a Random Forest classifier with 50 trees, minimum samples split of 10, and a fixed random state.
rf_model = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

# Train the Random Forest classifier on the training data using specified predictors.
rf_model = rf_model.fit(X_train, y_train)

In [13]:
# Generate predictions using the trained Random Forest classifier on the test data using specified predictors.
y_preds = rf_model.predict(X_test)

In [14]:
# measurements of model accuracy
acc = accuracy_score(y_test, y_preds)
acc

0.44047619047619047

In [15]:
# Create a contingency table showing the counts of actual versus predicted labels
combined = pd.DataFrame({"actual": y_test, "prediction": y_preds})
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,-1.0,0.0,1.0
actual,,,
-1.0,227,43,151
0.0,107,32,131
1.0,138,41,222


In [16]:
# Generate classification report
print(classification_report(y_test, y_preds))


              precision    recall  f1-score   support

        -1.0       0.48      0.54      0.51       421
         0.0       0.28      0.12      0.17       270
         1.0       0.44      0.55      0.49       401

    accuracy                           0.44      1092
   macro avg       0.40      0.40      0.39      1092
weighted avg       0.42      0.44      0.42      1092



In [17]:
# Group the matches DataFrame by the "team" column.
grouped_matches = matches.groupby("team")

In [18]:
# Get the group of matches specifically for the team "Liverpool".
group = grouped_matches.get_group("Liverpool")
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,0.0,0,0,2024,Liverpool,0,7,16,6,0.0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,1.0,0,1,2024,Liverpool,1,2,15,5,1.0
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,1.0,0,0,2024,Liverpool,0,18,16,6,1.0
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,0.0,0,0,2024,Liverpool,1,1,14,6,1.0
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,0.0,0,0,2024,Liverpool,0,27,12,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,2019-04-14,16:30,Premier League,Matchweek 34,Sun,Home,W,2.0,0.0,Chelsea,...,0.0,0,0,2019,Liverpool,1,7,16,6,1.0
46,2019-04-21,16:00,Premier League,Matchweek 35,Sun,Away,W,2.0,0.0,Cardiff City,...,1.0,1,1,2019,Liverpool,0,6,16,6,1.0
47,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Home,W,5.0,0.0,Huddersfield,...,0.0,0,0,2019,Liverpool,1,11,20,4,1.0
49,2019-05-04,19:45,Premier League,Matchweek 37,Sat,Away,W,3.0,2.0,Newcastle Utd,...,0.0,0,0,2019,Liverpool,0,18,19,5,1.0


In [19]:
# Calculate rolling averages for specified columns within a group.

def rolling_avg(group, cols, new_cols):
    """
    Args:
    - group: DataFrame group corresponding to matches of a particular team.
    - cols: List of columns for which rolling averages need to be calculated.
    - new_cols: List of new column names for the rolling average statistics.

    Returns:
    - DataFrame with the calculated rolling averages added as new columns.
    """
    # Sort the group by date to ensure chronological order.
    group = group.sort_values("date")

    # Calculate rolling averages for specified columns with a window size of 3.
    rolling_stats = group[cols].rolling(10, closed='left').mean()

    # Add the calculated rolling averages as new columns to the DataFrame.
    group[new_cols] = rolling_stats

    # Drop rows with NaN values resulting from the rolling window calculation.
    group= group.dropna(subset=new_cols)
    return group

In [20]:
# Define the columns for which rolling averages will be calculated.
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]

# Generate new column names for the corresponding rolling averages.
new_cols = [f"{c}_rolling" for c in cols]

new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [21]:
# Calculate rolling averages for cols in new_cols within the 'group' DataFrame.
rolling_avg(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
14,2018-11-03,17:30,Premier League,Matchweek 11,Sat,Away,D,1.0,1.0,Arsenal,...,5,0.0,2.0,0.4,14.4,5.1,17.12,0.7,0.1,0.1
16,2018-11-11,12:00,Premier League,Matchweek 12,Sun,Home,W,2.0,0.0,Fulham,...,6,1.0,1.7,0.5,13.9,4.7,17.47,0.5,0.1,0.1
17,2018-11-24,15:00,Premier League,Matchweek 13,Sat,Away,W,3.0,0.0,Watford,...,5,1.0,1.7,0.5,14.5,4.9,17.50,0.7,0.0,0.0
19,2018-12-02,16:15,Premier League,Matchweek 14,Sun,Home,W,1.0,0.0,Everton,...,6,1.0,1.9,0.5,13.3,4.9,17.49,0.7,0.0,0.0
20,2018-12-05,19:45,Premier League,Matchweek 15,Wed,Away,W,3.0,1.0,Burnley,...,2,1.0,1.8,0.4,13.9,4.8,17.48,0.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Home,W,3.0,1.0,Burnley,...,5,1.0,2.4,1.1,20.4,7.6,18.29,0.4,0.1,0.3
37,2024-02-17,12:30,Premier League,Matchweek 25,Sat,Away,W,4.0,1.0,Brentford,...,5,1.0,2.3,0.9,20.3,7.4,17.94,0.3,0.1,0.3
38,2024-02-21,19:30,Premier League,Matchweek 26,Wed,Home,W,4.0,1.0,Luton Town,...,2,1.0,2.5,1.0,20.3,7.4,17.89,0.4,0.1,0.3
41,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,W,1.0,0.0,Nott'ham Forest,...,5,1.0,2.7,1.0,21.8,8.5,17.84,0.4,0.1,0.3


In [22]:
# Define a lambda function to calculate rolling averages for each team's matches data.
# The lambda function takes each group 'x', representing matches data for a specific team,
# and applies the 'rolling_avg' function to calculate rolling averages for specified columns.
# 'x' represents a DataFrame containing the matches data for a particular team.
matches_rolling = matches.groupby("team").apply(lambda x: rolling_avg(x, cols, new_cols))
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 15 2018-11-03  17:30  Premier League  Matchweek 11   
                        17 2018-11-11  16:30  Premier League  Matchweek 12   
                        18 2018-11-25  13:30  Premier League  Matchweek 13   
                        20 2018-12-02  14:05  Premier League  Matchweek 14   
                        21 2018-12-05  20:00  Premier League  Matchweek 15   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 28 2024-02-10  15:00  Premier League  Matchweek 24   
                        29 2024-02-17  15:00  Premier League  Matchweek 25   
                        30 2024-02-25  13:30  Premier League  Matchweek 26   
                        32 2024-03-02  15:00  Premier League  Matchweek 27   
                        33 2024-03-09  15:00  Premier League  Matchweek 28   

                            day venue result   gf   ga        opponent  ...  \
team                                                                    ...   
Arsenal                 15  Sat  Home      D  1.0  1.0       Liverpool  ...   
                        17  Sun  Home      D  1.0  1.0          Wolves  ...   
                        18  Sun  Away      W  2.0  1.0     Bournemouth  ...   
                        20  Sun  Home      W  4.0  2.0       Tottenham  ...   
                        21  Wed  Away      D  2.0  2.0  Manchester Utd  ...   
...                         ...   ...    ...  ...  ...             ...  ...   
Wolverhampton Wanderers 28  Sat  Home      L  0.0  2.0       Brentford  ...   
                        29  Sat  Away      W  2.0  1.0       Tottenham  ...   
                        30  Sun  Home      W  1.0  0.0   Sheffield Utd  ...   
                        32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   
                        33  Sat  Home      W  2.0  1.0          Fulham  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 15         5     0.0         2.4         1.3   
                        17         6     0.0         2.5         1.2   
                        18         6     1.0         2.4         1.0   
                        20         6     1.0         2.3         1.0   
                        21         2     0.0         2.4         1.0   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 28         5    -1.0         1.9         1.4   
                        29         5     1.0         1.8         1.4   
                        30         6     1.0         1.9         1.5   
                        32         5    -1.0         1.9         1.4   
                        33         5     1.0         1.9         1.4   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 15       12.3         5.4        17.54        0.6   
                        17       12.6         5.5        17.20        0.6   
                        18       12.1         5.2        17.55        0.6   
                        20       12.4         4.6        17.44        0.5   
                        21       12.8         4.0        17.56        0.5   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 28       11.0         4.5        16.20        0.5   
                        29       12.1         4.7        16.84        0.6   
                        30       12.6         5.0        16.85        0.5   
                        32       12.9         4.8        16.79        0.5   
                        33       12.7         4.8        16.33        0.5   

                            pk_rolling  pkatt_rolli

In [23]:
# Drop the 'team' level from the index of the 'matches_rolling' DataFrame.
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
15,2018-11-03,17:30,Premier League,Matchweek 11,Sat,Home,D,1.0,1.0,Liverpool,...,5,0.0,2.4,1.3,12.3,5.4,17.54,0.6,0.0,0.0
17,2018-11-11,16:30,Premier League,Matchweek 12,Sun,Home,D,1.0,1.0,Wolves,...,6,0.0,2.5,1.2,12.6,5.5,17.20,0.6,0.0,0.0
18,2018-11-25,13:30,Premier League,Matchweek 13,Sun,Away,W,2.0,1.0,Bournemouth,...,6,1.0,2.4,1.0,12.1,5.2,17.55,0.6,0.0,0.0
20,2018-12-02,14:05,Premier League,Matchweek 14,Sun,Home,W,4.0,2.0,Tottenham,...,6,1.0,2.3,1.0,12.4,4.6,17.44,0.5,0.0,0.0
21,2018-12-05,20:00,Premier League,Matchweek 15,Wed,Away,D,2.0,2.0,Manchester Utd,...,2,0.0,2.4,1.0,12.8,4.0,17.56,0.5,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Home,L,0.0,2.0,Brentford,...,5,-1.0,1.9,1.4,11.0,4.5,16.20,0.5,0.2,0.2
29,2024-02-17,15:00,Premier League,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,...,5,1.0,1.8,1.4,12.1,4.7,16.84,0.6,0.2,0.2
30,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1.0,1.9,1.5,12.6,5.0,16.85,0.5,0.2,0.2
32,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,-1.0,1.9,1.4,12.9,4.8,16.79,0.5,0.2,0.2


In [24]:
# Reset the index of the 'matches_rolling' DataFrame to range from 0 to the number of rows.
# This reindexes the DataFrame with a simple integer index, replacing the previous index.
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2018-11-03,17:30,Premier League,Matchweek 11,Sat,Home,D,1.0,1.0,Liverpool,...,5,0.0,2.4,1.3,12.3,5.4,17.54,0.6,0.0,0.0
1,2018-11-11,16:30,Premier League,Matchweek 12,Sun,Home,D,1.0,1.0,Wolves,...,6,0.0,2.5,1.2,12.6,5.5,17.20,0.6,0.0,0.0
2,2018-11-25,13:30,Premier League,Matchweek 13,Sun,Away,W,2.0,1.0,Bournemouth,...,6,1.0,2.4,1.0,12.1,5.2,17.55,0.6,0.0,0.0
3,2018-12-02,14:05,Premier League,Matchweek 14,Sun,Home,W,4.0,2.0,Tottenham,...,6,1.0,2.3,1.0,12.4,4.6,17.44,0.5,0.0,0.0
4,2018-12-05,20:00,Premier League,Matchweek 15,Wed,Away,D,2.0,2.0,Manchester Utd,...,2,0.0,2.4,1.0,12.8,4.0,17.56,0.5,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Home,L,0.0,2.0,Brentford,...,5,-1.0,1.9,1.4,11.0,4.5,16.20,0.5,0.2,0.2
4062,2024-02-17,15:00,Premier League,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,...,5,1.0,1.8,1.4,12.1,4.7,16.84,0.6,0.2,0.2
4063,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1.0,1.9,1.5,12.6,5.0,16.85,0.5,0.2,0.2
4064,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,-1.0,1.9,1.4,12.9,4.8,16.79,0.5,0.2,0.2


In [25]:
def make_predictions(matches_rolling, predictors):
    """
    Make predictions using a random forest classifier.

    Args:
    - matches_rolling: DataFrame containing the rolling avg dataset.
    - predictors: List of column names used as predictors for the model.

    Returns:
    - combined: DataFrame containing actual and predicted labels.
    - cr: Classification report containing precision, recall, F1-score, and support.
    """

    # Split the dataset into features (X) and target variable (y)
    X = matches_rolling[predictors]
    y = matches_rolling["target"]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    # Train the random forest classifier on the training data
    rf_model.fit(X_train, y_train)
    # Generate predictions on the test data
    y_preds = rf_model.predict(X_test)
    # Create a DataFrame to combine actual and predicted labels
    combined = pd.DataFrame({"actual": y_test, "prediction": y_preds})
    # Generate a classification report
    cr = classification_report(y_test, y_preds)
    return combined, cr

In [26]:
# Make predictions using the random forest classifier on the 'matches_rolling' dataset,
# with predictors including both original features and new columns for rolling averages.
combined, cr = make_predictions(matches_rolling, predictors + new_cols)
print(cr)

              precision    recall  f1-score   support

        -1.0       0.50      0.62      0.56       396
         0.0       0.28      0.05      0.09       238
         1.0       0.47      0.60      0.53       383

    accuracy                           0.48      1017
   macro avg       0.42      0.42      0.39      1017
weighted avg       0.44      0.48      0.44      1017



In [27]:
# Merge additional match information from the 'matches_rolling' DataFrame into the 'combined' DataFrame.
# This additional information includes the date, team, opponent, and result of each match.
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined

,actual,prediction,date,team,opponent,result
3884,1.0,1.0,2019-05-04,Wolverhampton Wanderers,Fulham,W
2841,1.0,1.0,2022-04-23,Newcastle United,Norwich City,W
1907,-1.0,-1.0,2019-05-06,Leicester City,Manchester City,L
3489,0.0,-1.0,2023-08-13,Tottenham Hotspur,Brentford,D
2433,1.0,1.0,2022-08-27,Manchester City,Crystal Palace,W
...,...,...,...,...,...,...
725,-1.0,1.0,2022-02-26,Brighton and Hove Albion,Aston Villa,L
2379,1.0,1.0,2021-02-27,Manchester City,West Ham,W
3007,1.0,-1.0,2023-08-18,Nottingham Forest,Sheffield Utd,W
282,-1.0,-1.0,2021-10-22,Aston Villa,Arsenal,L


In [28]:
# Define a custom dictionary class to handle missing keys by returning the key itself.
# This class ensures that if a key is not found in the dictionary, it returns the key itself.
class MissingDict(dict):
    __missing__ = lambda self, key: key

# Define a dictionary containing mappings of team names to their shortened versions.
map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchested Utd",
    "Newcastle United": "Newcastle Utd",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

# Create an instance of the MissingDict class with the provided mapping.
mapping = MissingDict(**map_values)

In [29]:
# Map team names in the 'team' column of the 'combined' DataFrame to their shortened versions
# using the custom dictionary 'mapping' created earlier, and store the results in a new column 'new_team'.
combined["new_team"] = combined ["team"].map(mapping)
combined

,actual,prediction,date,team,opponent,result,new_team
3884,1.0,1.0,2019-05-04,Wolverhampton Wanderers,Fulham,W,Wolves
2841,1.0,1.0,2022-04-23,Newcastle United,Norwich City,W,Newcastle Utd
1907,-1.0,-1.0,2019-05-06,Leicester City,Manchester City,L,Leicester City
3489,0.0,-1.0,2023-08-13,Tottenham Hotspur,Brentford,D,Tottenham Hotspur
2433,1.0,1.0,2022-08-27,Manchester City,Crystal Palace,W,Manchester City
...,...,...,...,...,...,...,...
725,-1.0,1.0,2022-02-26,Brighton and Hove Albion,Aston Villa,L,Brighton
2379,1.0,1.0,2021-02-27,Manchester City,West Ham,W,Manchester City
3007,1.0,-1.0,2023-08-18,Nottingham Forest,Sheffield Utd,W,Nottingham Forest
282,-1.0,-1.0,2021-10-22,Aston Villa,Arsenal,L,Aston Villa


In [30]:
# Merge the 'combined' DataFrame with itself based on matching dates and team/opponent pairs.
# This operation matches rows where the 'date' column and 'new_team' column in the left DataFrame
# match the 'date' column and 'opponent' column in the right DataFrame, respectively.
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,-1.0,-1.0,2019-05-06,Leicester City,Manchester City,L,Leicester City,1.0,1.0,Manchester City,Leicester City,W,Manchester City
1,-1.0,-1.0,2021-08-22,Wolverhampton Wanderers,Tottenham,L,Wolves,1.0,1.0,Tottenham Hotspur,Wolves,W,Tottenham Hotspur
2,1.0,-1.0,2022-03-13,West Ham United,Aston Villa,W,West Ham,-1.0,-1.0,Aston Villa,West Ham,L,Aston Villa
3,0.0,-1.0,2021-10-02,Everton,Manchester Utd,D,Everton,0.0,1.0,Manchester United,Everton,D,Manchested Utd
4,-1.0,-1.0,2018-11-03,Burnley,West Ham,L,Burnley,1.0,-1.0,West Ham United,Burnley,W,West Ham
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,1.0,1.0,2022-05-01,Arsenal,West Ham,W,Arsenal,-1.0,-1.0,West Ham United,Arsenal,L,West Ham
191,-1.0,-1.0,2022-02-19,Brentford,Arsenal,L,Brentford,1.0,1.0,Arsenal,Brentford,W,Arsenal
192,-1.0,-1.0,2023-04-01,Liverpool,Manchester City,L,Liverpool,1.0,1.0,Manchester City,Liverpool,W,Manchester City
193,0.0,-1.0,2020-11-28,Brighton and Hove Albion,Liverpool,D,Brighton,0.0,1.0,Liverpool,Brighton,D,Liverpool


In [31]:
# Filter the 'merged' DataFrame to select rows where one prediction is a win (1)
# and the other prediction is a loss (-1)
# then count the occurrences of actual labels for those filtered rows.
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == -1)]["actual_x"].value_counts()

actual_x
 1.0    31
 0.0    13
-1.0    11
Name: count, dtype: int64